# SENA demo

In [ ]:
# imports
import json
import logging
import os
import sys
import pickle
import random
import torch
import numpy as np
import pandas as pd
from dataclasses import asdict, dataclass, field
from typing import Any, Optional, List
import scanpy as sc
from torch.utils.data import DataLoader, TensorDataset
from scipy.stats import pearsonr


# SENA repo imports
# Found in ../SENA/src
from utils import Norman2019DataLoader # review Acuerdate de añadir utils a la carpeta de sena
from train import train

# Own sena-related imports
# Importable from pip install -e ../sypp
from sena.sena_utils import check_and_load_paths
from data_utils.metrics import MMDLoss, compute_kld
from sena.sena_utils import find_pert_pairs


In [3]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("training.log"),  # Logs to a file
        logging.StreamHandler(),  # Also logs to the console
    ],
)

In [4]:
@dataclass
class Options:
    name: str = "demo_test" # args
    model: str = "sena"
    dataset_name: str = "Norman2019_reduced" # args
    batch_size: int = 32
    sena_lambda: float = 0 # args
    lr: float = 1e-3
    epochs: int = 30 # args
    grad_clip: bool = False
    mxAlpha: float = 1.0
    mxBeta: float = 1.0
    mxTemp: float = 100.0
    lmbda: float = 0.1
    MMD_sigma: float = 200.0
    kernel_num: int = 10
    matched_IO: bool = False
    latdim: int = 105 # args
    seed: int = 42 # args
    num_runs: int = 1 # args
    dim: Optional[int] = None
    cdim: Optional[int] = None
    log: bool = False
    mlflow_port: int = 5678
    sena_eval_mode: List[str] = field(default_factory=lambda: ["double"]) # args
    numint: int = 2
    pool_size: int = 50 # args

## Some code

In [5]:
def set_seeds(seed: int) -> None:
    """Set random seeds for reproducibility."""
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [6]:
# Set up configuration for logging
def save_config(opts: Options, save_dir: str) -> None:
    """Save the configuration options to a JSON file."""
    config_path = os.path.join(save_dir, "config.json")
    with open(config_path, "w") as f:
        json.dump(asdict(opts), f, indent=4)

In [7]:
# Save dataloader as pickle
def save_pickle(data: Any, filepath: str) -> None:
    """Save data to a pickle file."""
    with open(filepath, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
def compute_metrics(gt_x_list, pred_x_list, gt_y_list,
                    pred_y_list, c_y_list, mu_list, var_list,
                    MSE_pred_true_temp, KLD_pred_true_temp, MMD_pred_true_temp,
                    MSE_ctrl_true_temp, KLD_ctrl_true_temp, MMD_ctrl_true_temp,
                    mmd_loss_func):
    """
    asdf
    """
    gt_x = torch.vstack(gt_x_list)
    pred_x = torch.vstack(pred_x_list)
    gt_y = torch.vstack(gt_y_list)
    pred_y = torch.vstack(pred_y_list)
    c_y = torch.vstack(c_y_list)
    mu = torch.vstack(mu_list)
    var = torch.vstack(var_list)

    # Compute MSE
    MSE_pred_true = torch.mean((gt_y - pred_y)**2)
    MSE_ctrl_true = torch.mean((gt_y - gt_x)**2)

    # Compute MMD
    MMD_pred_true = mmd_loss_func(gt_y, pred_y)
    MMD_ctrl_true = mmd_loss_func(gt_y, gt_x)

    # Compute KLD
    KLD_pred_true = compute_kld(gt_y, pred_y)
    KLD_ctrl_true = compute_kld(gt_y, gt_x)


    # Save temporal calculations
    MSE_pred_true_temp.append(MSE_pred_true.item())
    MSE_ctrl_true_temp.append(MSE_ctrl_true.item())

    MMD_pred_true_temp.append(MMD_pred_true.item())
    MMD_ctrl_true_temp.append(MMD_ctrl_true.item())

    KLD_pred_true_temp.append(KLD_pred_true.item())
    KLD_ctrl_true_temp.append(KLD_ctrl_true.item())

    return (gt_x_list, pred_x_list, gt_y_list,
            pred_y_list, c_y_list, mu_list, var_list,
            MSE_pred_true_temp, KLD_pred_true_temp, MMD_pred_true_temp,
            MSE_ctrl_true_temp, KLD_ctrl_true_temp, MMD_ctrl_true_temp)

In [9]:
def evaluate_double(model,
                    dataloader,
                    data_path,
                    ptb_genes,
                    config,
                    results_dir_path: str,
                    device: str,
                    run_name: str,
                    numint: int = 2) -> None:
    """
    W
    """
    # Load config
    pool_size = config.get("pool_size", 150)
    MMD_sigma = config.get("MMD_sigma", 200.0)
    kernel_num = config.get("kernel_num", 10)
    # matched_IO = config.get("matched_IO", False)
    temp = config.get("temp", 1000.0)
    seed = config.get("seed", 42)
    # latdim = config.get("latdim", 105)
    # model_name = config.get("name", "example")
    batch_size = 5
    top_deg = 20

    set_seeds(seed)

    # Load raw data, Norman for now
    if os.path.exists(data_path):
        adata = sc.read_h5ad(data_path)
    else:
        raise FileNotFoundError(f"Data file not found at {data_path}")


    # Only keeps the matrix
    ctrl_adata = adata[adata.obs["condition"] == "ctrl"].X.copy()
    baseline_ctrl = np.mean(ctrl_adata.toarray(), axis=0)

    # Load double perturbation data
    ptb_targets = sorted(adata.obs["guide_ids"].unique().tolist())[1:]
    double_adata = adata[
        (adata.obs["guide_ids"].str.contains(","))
        & (
            adata.obs["guide_ids"].map(
                lambda x: all([y in ptb_targets for y in x.split(",")])
            )
        )
    ]

    # Randomply sampling same subset of control data based on seed
    ctrl_random = ctrl_adata[np.random.choice(ctrl_adata.shape[0], pool_size, replace=True)]

    print("Proceeding with evaluation of intervened pairs.")
    # Make results file path.
    results_file_path = os.path.join(
        results_dir_path, f"{run_name}_double_metrics.csv"
        )

    # Finding perturbation pairs for model and shape of c (for each dataloader)
    all_pairs, _, c_shape = find_pert_pairs(dataloader=dataloader, device=device)

    # Prepare writing results to file
    with open(file=results_file_path, mode="w") as f:
            print(
                f"double,mmd_true_vs_ctrl,mmd_true_vs_pred,mse_true_vs_ctrl,mse_true_vs_pred,kld_true_vs_ctrl,kld_true_vs_pred,PearsonTop{top_deg}_true_vs_ctrl,Pearson_pval_true_vs_ctrl,PearsonTop{top_deg}_true_vs_pred,Pearson_pval_true_vs_pred",
                file=f,
            )

            # Initialise lists  
            pred_x_list, gt_x_list = [], []
            gt_y_list, pred_y_list = [], []
            c_y_list, mu_list, var_list = [], [], []
            MSE_pred_true_temp, KLD_pred_true_temp, MMD_pred_true_temp = [], [], []
            MSE_ctrl_true_temp, KLD_ctrl_true_temp, MMD_ctrl_true_temp = [], [], []
            full_preds = []
            predictions_list = [] # initialise dataframe to save mean predictions for each double

            # Initialise MMD function
            mmd_loss_func = MMDLoss(fix_sigma=MMD_sigma, kernel_num=kernel_num)

            for num, unique_pairs in enumerate(all_pairs):
                gene_pair = "+".join([ptb_genes[unique_pairs[0]], ptb_genes[unique_pairs[1]]])
                print(f"Evaluating for intervention pair {num+1}/{len(all_pairs)}: {gene_pair}")

                # Turning control samples into tensor
                ctrl_geps_tensor = torch.tensor(ctrl_random.toarray()).double()
                # ctrl_geps_tensor = torch.tensor(ctrl_random.toarray())

                # Get equivalent double adata or random sample
                unique_pairs_set = set([ptb_genes[unique_pairs[0]], ptb_genes[unique_pairs[1]]])
                double_samples = double_adata[double_adata.obs["guide_ids"].map(
                     lambda x: all([y in unique_pairs_set for y in x.split(",")])
                )]

                if double_samples.n_obs == 0:
                    print(f"0 samples found for {gene_pair}. Taking random samples from whole adata.")
                    true_geps = adata.copy()
                    random_indices = np.random.choice(
                        adata.n_obs, size=pool_size, replace=True
                        )
                    true_geps = true_geps[random_indices, :]

                elif double_samples.n_obs < pool_size:
                    print(f"Warning: Not enough samples for {gene_pair}. Randomly selecting samples from {double_samples.n_obs} samples.")
                    true_geps = double_samples.copy()
                    true_geps = true_geps[np.random.choice(double_samples.n_obs, size=pool_size, replace=True), :]

                else:
                    random_indices = np.random.choice(double_samples.n_obs, size=pool_size, replace=False)
                    true_geps = double_samples[random_indices, :]

                # Turning true samples from predictions into tensor
                baseline_true = np.mean(true_geps.X.toarray(), axis=0)
                true_geps_tensor = torch.tensor(true_geps.X.toarray()).double()

                # Saving both in same Tensor Dataset
                ctrl_double_dataset = TensorDataset(ctrl_geps_tensor, true_geps_tensor)
                ctrl_double_loader = DataLoader(ctrl_double_dataset, batch_size=32, shuffle=False)

                for i, (x, y) in enumerate(ctrl_double_loader):
                    x = x.to(device)
                    y = y.to(device)

                    c_shape_loader = c_shape[0, :].repeat(32*batch_size, 1)
                    c1 = torch.zeros_like(c_shape_loader).double().to(device)
                    c1[:, unique_pairs[0]] = 1
                    c2 = torch.zeros_like(c_shape_loader).double().to(device)
                    c2[:, unique_pairs[1]] = 1

                    if len(x) < len(c1):
                        c1 = c1[:len(x), :]
                        c2 = c2[:len(x), :]

                    with torch.no_grad():
                        y_hat, x_recon, z_mu, z_var, _, _ = model(
                            x, c1, c2, num_interv=numint, temp=temp)

                    gt_x_list.append(x.cpu())
                    pred_x_list.append(x_recon.cpu())

                    gt_y_list.append(y.cpu())
                    pred_y_list.append(y_hat.cpu())
                    full_preds.append(y_hat.cpu())

                    c_y_list.append(c_shape.cpu())
                    mu_list.append(z_mu.cpu())
                    var_list.append(z_var.cpu())


                    # Limit stacked tensors while iterating through desired indices
                    if len(gt_x_list) >= batch_size:

                        (gt_x_list, pred_x_list, gt_y_list,
                        pred_y_list, c_y_list, mu_list, var_list,
                        MSE_pred_true_temp, KLD_pred_true_temp, MMD_pred_true_temp,
                        MSE_ctrl_true_temp, KLD_ctrl_true_temp, MMD_ctrl_true_temp) = compute_metrics(gt_x_list, pred_x_list, gt_y_list,
                                                                                                    pred_y_list, c_y_list, mu_list, var_list,
                                                                                                    MSE_pred_true_temp, KLD_pred_true_temp, MMD_pred_true_temp,
                                                                                                    MSE_ctrl_true_temp, KLD_ctrl_true_temp, MMD_ctrl_true_temp,
                                                                                                    mmd_loss_func)

                        # Reset lists
                        pred_x_list, gt_x_list = [], []
                        gt_y_list, pred_y_list = [], []
                        c_y_list, mu_list, var_list = [], [], []

                # Once iterated through all indices, check rest of stacked tensors
                if len(gt_x_list) > 0:

                    (gt_x_list, pred_x_list, gt_y_list,
                    pred_y_list, c_y_list, mu_list, var_list,
                    MSE_pred_true_temp, KLD_pred_true_temp, MMD_pred_true_temp,
                    MSE_ctrl_true_temp, KLD_ctrl_true_temp, MMD_ctrl_true_temp) = compute_metrics(gt_x_list, pred_x_list, gt_y_list,
                                                                                                pred_y_list, c_y_list, mu_list, var_list,
                                                                                                MSE_pred_true_temp, KLD_pred_true_temp, MMD_pred_true_temp,
                                                                                                MSE_ctrl_true_temp, KLD_ctrl_true_temp, MMD_ctrl_true_temp,
                                                                                                mmd_loss_func)


                # After each unique pair, print and save metrics
                print(f"MMD (true vs. control):     {np.mean(MMD_ctrl_true_temp):.6f}")
                print(f"MMD (true vs. predicted):   {np.mean(MMD_pred_true_temp):.6f}")
                print(f"MSE (true vs. control):     {np.mean(MSE_ctrl_true_temp):.6f}")
                print(f"MSE (true vs. predicted):   {np.mean(MSE_pred_true_temp):.6f}")
                print(f"KLD (true vs. control):     {np.mean(KLD_ctrl_true_temp):.6f}")
                print(f"KLD (true vs. predicted):   {np.mean(KLD_pred_true_temp):.6f}")

                # Calculating Pearson with mean GEPs
                mean_pred = torch.vstack(full_preds).mean(dim=0).cpu()
                true_deg = baseline_true-baseline_ctrl
                pred_deg = mean_pred.numpy()-baseline_ctrl
                ctrl_ctrl_deg = baseline_ctrl-baseline_ctrl
                deg_idx = np.argsort(abs(true_deg))[-top_deg:] # DEG from true pert value

                pred_true_pearson_stat,  pred_true_pearson_pval = pearsonr(true_deg[deg_idx], pred_deg[deg_idx])
                ctrl_true_pearson_stat,  ctrl_true_pearson_pval = pearsonr(true_deg[deg_idx], ctrl_ctrl_deg[deg_idx])

                print(f"Pearson Top {top_deg} DEG (true vs. control): {ctrl_true_pearson_stat:.6f} | p-value: {ctrl_true_pearson_pval:.6f}")
                print(f"Pearson Top {top_deg} DEG (true vs. predicted): {pred_true_pearson_stat:.6f} | p-value: {pred_true_pearson_pval:.6f}")

                # Save results to file
                print(f"{gene_pair},{np.mean(MMD_ctrl_true_temp)},{np.mean(MMD_pred_true_temp)},{np.mean(MSE_ctrl_true_temp)},{np.mean(MSE_pred_true_temp)},{np.mean(KLD_ctrl_true_temp)},{np.mean(KLD_pred_true_temp)},{ctrl_true_pearson_stat},{ctrl_true_pearson_pval},{pred_true_pearson_stat},{pred_true_pearson_pval}",
                    file=f,
                )

                # Saving and updating predictions
                prediction_row = [gene_pair] + mean_pred.tolist()
                predictions_list.append(prediction_row)


                # Reset lists
                pred_x_list, gt_x_list = [], []
                gt_y_list, pred_y_list = [], []
                c_y_list, mu_list, var_list = [], [], []
                MSE_pred_true_temp, KLD_pred_true_temp, MMD_pred_true_temp = [], [], []
                MSE_ctrl_true_temp, KLD_ctrl_true_temp, MMD_ctrl_true_temp = [], [], []


            # Save prediction data
            prediction_colnames = ['double'] + adata.var_names.tolist()
            prediction_df = pd.DataFrame(predictions_list, columns=prediction_colnames)
            prediction_file_path = os.path.join(results_dir_path, f"{run_name}_double_prediction.csv")
            prediction_df.to_csv(prediction_file_path)
            print(f"Saved predictions at {prediction_file_path}")



# Still need to save predictions somewhere (forgot)

## SENA

Argument setup

In [10]:
savedir = '../cris_test/models/'
eval_dir = '../cris_test/results/'
name = 'demo_test'
opts = Options()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f"Configuration: {opts}")

# Build the save directory path
savedir = os.path.join(savedir, name)
os.makedirs(savedir, exist_ok=True)

eval_dir = os.path.join(eval_dir, name)
os.makedirs(eval_dir, exist_ok=True)

2025-06-30 15:29:52,893 - INFO - Configuration: Options(name='demo_test', model='sena', dataset_name='Norman2019_reduced', batch_size=32, sena_lambda=0, lr=0.001, epochs=30, grad_clip=False, mxAlpha=1.0, mxBeta=1.0, mxTemp=100.0, lmbda=0.1, MMD_sigma=200.0, kernel_num=10, matched_IO=False, latdim=105, seed=42, num_runs=1, dim=None, cdim=None, log=False, mlflow_port=5678, sena_eval_mode=['double'], numint=2, pool_size=50)


## Training

In [11]:
# Set up current seed for specific run and name for current run
current_run = 0
current_seed = opts.seed
run_name = f"{opts.name}_seed_{current_seed}"
set_seeds(current_seed)

# Update directory for each run
savedir = os.path.join(savedir, run_name)
os.makedirs(savedir, exist_ok=True)


logging.info(f"Loading {opts.dataset_name} dataset")
# Dataset choice
if 'Norman2019' in opts.dataset_name:
    data_handler = Norman2019DataLoader(batch_size=opts.batch_size, dataname=opts.dataset_name)

else:
    raise ValueError(f"Dataset {opts.dataset_name} is not supported yet.")

# Get data from single-gene perturbation
(
    dataloader,
    dataloader2,
    dim,
    cdim,
    ptb_targets,
) = data_handler.get_data(mode="train")

# Get data from double perturbation
dataloader_double, _, _, _ = data_handler.get_data(mode="test")

# Dimensions for SENA
opts.dim = dim
opts.cdim = cdim

# I think this is not needed, but I leave it here for now
if opts.latdim is None:
    opts.latdim = opts.cdim


2025-06-30 15:31:21,033 - INFO - Loading Norman2019_reduced dataset


In [12]:
# Save configurations and data
save_config(opts, savedir)
save_pickle(ptb_targets, os.path.join(savedir, "ptb_targets.pkl"))
save_pickle(dataloader2, os.path.join(savedir, "test_data_single_node.pkl"))
save_pickle(dataloader, os.path.join(savedir, "train_data.pkl"))
save_pickle(dataloader_double, os.path.join(savedir, "double_data.pkl"))

# Review: currently only for double
data_file_map = {
    "train": "train_data.pkl",
    "test": "test_data_single_node.pkl",
    "double": "double_data.pkl",
}

In [13]:
# Running process for SENA
print(f"Initialising SENA run: {current_run+1}, seed: {current_seed}")
train(dataloader=dataloader,
        opts=opts,
        device=device,
        savedir=savedir,
        logger=logging, # Review
        data_handler=data_handler)

logging.info(f"Training completed for run {current_run + 1} with seed {current_seed}")


2025-06-30 15:31:46,671 - INFO - Started training on device: cuda


Initialising SENA run: 1, seed: 42


2025-06-30 15:31:48,289 - INFO - Training for 30 epochs...
Epoch 1/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [00:05<00:00, 20.23batch/s]
2025-06-30 15:31:53,486 - INFO - Epoch 1: Loss=0.079592, MMD=3.961621, MSE=0.072350, KL=0.008230, L1=0.072413
2025-06-30 15:31:53,556 - INFO - Best model saved at epoch 1
Epoch 2/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [00:03<00:00, 32.74batch/s]
2025-06-30 15:31:56,767 - INFO - Epoch 2: Loss=0.040006, MMD=4.035532, MSE=0.033806, KL=0.025455, L1=0.062003
Epoch 3/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [00:03<00:00, 31.36batch/s]
2025-06-30 15:32:00,119 - INFO - Epoch 3: Loss=0.038498, MMD=4.503899, MSE=0.033384, KL=0.038342, L1=0.051149
Epoch 4/30: 100%|█████████████████████████████████████████████████████

## Predict -  Evaluate

In [14]:
# Predict and evaluate

# Load the model and data for evaluation
config_path = os.path.join(savedir, "config.json")
model_path = os.path.join(savedir, "best_model.pt")
# This is to load the appropriate pickle, 'double' by default
data_path = os.path.join(savedir, data_file_map[opts.sena_eval_mode[0]])
ptb_path = os.path.join(savedir, "ptb_targets.pkl")
logging.info(f"Model and data saved in {savedir}.")

# Loading required data from .pkl files for evaluation
(dataloader, model, ptb_genes, config) = check_and_load_paths(model_path=model_path, 
            data_path=data_path,
            ptb_path=ptb_path,
            config_path=config_path,
            mode=opts.sena_eval_mode,
            savedir=savedir)


2025-06-30 15:33:54,308 - INFO - Model and data saved in ../cris_test/models/demo_test/demo_test_seed_42.


In [18]:
# Preparations for evaluation
model = model.to(device)
model.eval()

# Given that it is still in SENA folder
raw_data_path = os.path.join('../SENA/data',f"{opts.dataset_name}.h5ad")

# Current model evaluation and saving metrics to the directory as CSV
evaluate_double(
    model=model,
    dataloader=dataloader,
    data_path=raw_data_path,
    config=config,
    ptb_genes=ptb_genes,
    results_dir_path=eval_dir,
    run_name=run_name,
    device=device,
    numint=opts.numint
)

print("Process end.")

Proceeding with evaluation of intervened pairs.


Finding intervention pairs: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 131/131 [00:01<00:00, 91.12it/s]


Evaluating for intervention pair 1/131: AHR+FEV
MMD (true vs. control):     0.866754
MMD (true vs. predicted):   0.891912
MSE (true vs. control):     0.118583
MSE (true vs. predicted):   0.126756
KLD (true vs. control):     0.090812
KLD (true vs. predicted):   0.096792
Pearson Top 20 DEG (true vs. control): nan | p-value: nan
Pearson Top 20 DEG (true vs. predicted): -0.393490 | p-value: 0.086076
Evaluating for intervention pair 2/131: AHR+KLF1


/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


MMD (true vs. control):     0.506677
MMD (true vs. predicted):   0.565586
MSE (true vs. control):     0.071022
MSE (true vs. predicted):   0.075731
KLD (true vs. control):     0.012072
KLD (true vs. predicted):   0.017185
Pearson Top 20 DEG (true vs. control): nan | p-value: nan
Pearson Top 20 DEG (true vs. predicted): 0.254020 | p-value: 0.279823
Evaluating for intervention pair 3/131: BAK1+BCL2L11


...

Pearson Top 20 DEG (true vs. control): nan | p-value: nan
Pearson Top 20 DEG (true vs. predicted): 0.485521 | p-value: 0.029996
Evaluating for intervention pair 130/131: UBASH3A+UBASH3B
MMD (true vs. control):     0.434944
MMD (true vs. predicted):   0.493677
MSE (true vs. control):     0.071933
MSE (true vs. predicted):   0.075035
KLD (true vs. control):     0.006123
KLD (true vs. predicted):   0.006115


/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Pearson Top 20 DEG (true vs. control): nan | p-value: nan
Pearson Top 20 DEG (true vs. predicted): 0.656947 | p-value: 0.001650
Evaluating for intervention pair 131/131: UBASH3B+ZBTB25
MMD (true vs. control):     0.533852
MMD (true vs. predicted):   0.575637
MSE (true vs. control):     0.076611
MSE (true vs. predicted):   0.081746
KLD (true vs. control):     0.016970
KLD (true vs. predicted):   0.015499


/opt/conda/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Pearson Top 20 DEG (true vs. control): nan | p-value: nan
Pearson Top 20 DEG (true vs. predicted): 0.626779 | p-value: 0.003103
Saved predictions at ../cris_test/results/demo_test/demo_test_seed_42_double_prediction.csv
Process end.


## Results

In [20]:
filepath = '../cris_test/results/demo_test/demo_test_seed_42_double_metrics.csv'
# filepath = '../cris_test/results/demo_test/demo_test_seed_42_double_prediction.csv'

results = pd.read_csv(filepath)

In [21]:
results

,double,mmd_true_vs_ctrl,mmd_true_vs_pred,mse_true_vs_ctrl,mse_true_vs_pred,kld_true_vs_ctrl,kld_true_vs_pred,PearsonTop20_true_vs_ctrl,Pearson_pval_true_vs_ctrl,PearsonTop20_true_vs_pred,Pearson_pval_true_vs_pred
0,AHR+FEV,0.866754,0.891912,0.118583,0.126756,0.090812,0.096792,NaN,NaN,-0.393490,8.607579e-02
1,AHR+KLF1,0.506677,0.565586,0.071022,0.075731,0.012072,0.017185,NaN,NaN,0.254020,2.798235e-01
2,BAK1+BCL2L11,0.392806,0.466893,0.066550,0.071575,0.003831,0.004644,NaN,NaN,0.708671,4.695582e-04
3,BAK1+KLF1,0.485175,0.603236,0.068348,0.072059,0.010004,0.010587,NaN,NaN,0.196517,4.063097e-01
4,BAK1+TMSB4X,0.422107,0.475627,0.069899,0.073665,0.025339,0.021995,NaN,NaN,0.766091,8.209391e-05
...,...,...,...,...,...,...,...,...,...,...,...
126,SNAI1+UBASH3B,0.454423,0.514041,0.075972,0.084358,0.009244,0.010687,NaN,NaN,0.895767,9.397110e-08
127,SNAI1+ZBTB10,0.771285,0.702781,0.114876,0.127682,0.035458,0.039644,NaN,NaN,0.560491,1.015346e-02
128,TBX2+TBX3,0.552896,0.575162,0.079332,0.082652,0.014430,0.015578,NaN,NaN,0.485521,2.999571e-02
129,UBASH3A+UBASH3B,0.434944,0.493677,0.071933,0.075035,0.006123,0.006115,NaN,NaN,0.656947,1.650200e-03
